### 贈與稅法

In [1]:
from z3 import Solver, Real, sat, Implies, Or, And, If, ToInt

def gift_tax_calculator(
    # period_choice 代表使用者選擇之贈與日:
    # (1) 民國98年1月23日至106年5月11日
    # (2) 民國106年5月12日至110年12月31日
    # (3) 民國111年1月1日(含)至民國113年12月31日
    # (4) 民國114年1月1日(含)以後
    period_choice: int,

    # ====== 以下對應「本次贈與總額 (1+2+3+4)」 ======
    # 1.土地(以公告土地現值或評定標準價格為準)_元
    land_value,  
    # 2.地上物(以公告土地現值或評定標準價格為準)_元
    ground_value,
    # 3.房屋(以評定標準價格為準)_元
    house_value,
    # 4.動產及其他有財產價值的權利_元
    others_value,

    # ====== 四.不計入贈與總額之財產(1+2+3) ======
    #   1.土地_元
    #   2.房屋_元
    #   3.動產及其他有財產價值的權利_元
    not_included_land,
    not_included_house,
    not_included_others,

    # ====== (98 年舊制時需要使用者直接輸入的「本年度內剩餘尚未使用之免稅額」) ======
    remaining_exemption_98=0,

    # ====== (106/111/114 年，需同一年內先前贈與總額，用於合併計算) ======
    previous_gift_sum_in_this_year=0,

    # ====== 三.扣除額 (1+2+3) ======
    #   1.土地增值稅_元
    #   2.契稅_元
    #   3.其他贈與負擔_元
    land_increment_tax=0,
    deed_tax=0,
    other_gift_burdens=0,

    # ====== 五.同一年內以前各次應納稅額及可扣抵稅額 ======
    previous_gift_tax_or_credit=0,

    # ====== 六.「新舊制差額調整」(主要用於106年) ======
    new_old_system_adjustment=0
):
    """
    回傳值: 本次應納贈與稅額(元)（整數）
    若無法求解(unsat)則回傳 None。

    特別說明：
      - period_choice=1 (98年舊制)：直接輸入 remaining_exemption_98。
      - period_choice=2 (106年)：免稅額固定220萬，新舊制差額調整如有即輸入。
      - period_choice=3 (111年)：免稅額固定244萬，採用111年之計算方式。
      - period_choice=4 (114年以後)：免稅額同111年(244萬)，但稅率與累進差額改為：
            若課稅贈與淨額 ≤ 28,110,000：稅率 10%，累進差額 0；
            若 28,110,001 ≤ 課稅贈與淨額 ≤ 56,210,000：稅率 15%，累進差額 1,405,500；
            若 課稅贈與淨額 ≥ 56,210,001：稅率 20%，累進差額 4,216,000。
    """

    solver = Solver()

    # ------------------
    # 定義 Z3 變數
    # ------------------
    # (A) 本次贈與總額 (扣除「不計入贈與總額之財產」)
    this_gift_total = Real('this_gift_total')

    # (B) 年度(累計)贈與總額 (本次 + 以前各次)
    annual_gift_total = Real('annual_gift_total')

    # (C) 年度課稅贈與淨額（或本次課稅贈與淨額）
    annual_taxable_net = Real('annual_taxable_net')

    # (D) 最終要求解之「本次應納贈與稅額」
    gift_tax_result = Real('gift_tax_result')

    # (E) 三段稅率及累進差額
    applied_rate = Real('applied_rate')
    progressive_diff = Real('progressive_diff')

    # (F) 扣除額
    total_deductions = Real('total_deductions')

    # ------------------
    # 參數約束
    # ------------------
    # 當次贈與總額（不扣除不計入部分）
    solver.add(
        this_gift_total
        == (land_value + ground_value + house_value + others_value)
    )

    # 年度累計贈與總額 (本次 + 以前)
    solver.add(annual_gift_total == this_gift_total + previous_gift_sum_in_this_year)

    # 扣除額 = 土地增值稅 + 契稅 + 其他贈與負擔
    solver.add(total_deductions == (land_increment_tax + deed_tax + other_gift_burdens))

    # ---------------------------
    # 根據不同期間選擇計算方式
    # ---------------------------
    if period_choice == 1:
        #
        # (1) 98年1月23日 ~ 106年5月11日 舊制
        #
        # 使用者輸入剩餘免稅額 remaining_exemption_98
        this_taxable_net = Real('this_taxable_net')
        solver.add(
            this_taxable_net
            == this_gift_total - remaining_exemption_98 - total_deductions
        )
        # 稅率 10%，無累進差額
        solver.add(applied_rate == 0.10)
        solver.add(progressive_diff == 0)
        # 應納稅額計算
        solver.add(
            gift_tax_result
            == (this_taxable_net * applied_rate) - previous_gift_tax_or_credit
        )

    elif period_choice == 2:
        #
        # (2) 106年5月12日 ~ 110年12月31日
        #
        exemption_amount = 2200000
        solver.add(
            annual_taxable_net
            == annual_gift_total - exemption_amount - total_deductions
        )
        # 三段稅率與累進差額：
        # (1) ≤25,000,000元：10% & 0
        # (2) 25,000,001~50,000,000元：15% & 1,250,000
        # (3) >50,000,000元：20% & 3,750,000
        cond1 = (annual_taxable_net <= 25000000)
        cond2 = And(annual_taxable_net > 25000000, annual_taxable_net <= 50000000)
        cond3 = (annual_taxable_net > 50000000)
        solver.add(Or(cond1, cond2, cond3))
        solver.add(Implies(cond1, And(applied_rate == 0.10, progressive_diff == 0)))
        solver.add(Implies(cond2, And(applied_rate == 0.15, progressive_diff == 1250000)))
        solver.add(Implies(cond3, And(applied_rate == 0.20, progressive_diff == 3750000)))
        solver.add(
            gift_tax_result
            == (annual_taxable_net * applied_rate)
               - progressive_diff
               - previous_gift_tax_or_credit
               - new_old_system_adjustment
        )

    elif period_choice == 3:
        #
        # (3) 111年1月1日(含)至民國113年12月31日
        #
        exemption_amount = 2440000
        solver.add(
            annual_taxable_net
            == annual_gift_total - exemption_amount - total_deductions
        )
        # 稅率與累進差額：
        # (1) ≤25,000,000元：10% & 0
        # (2) 25,000,001~50,000,000元：15% & 1,250,000
        # (3) >50,000,000元：20% & 3,750,000
        cond1 = (annual_taxable_net <= 25000000)
        cond2 = And(annual_taxable_net > 25000000, annual_taxable_net <= 50000000)
        cond3 = (annual_taxable_net > 50000000)
        solver.add(Or(cond1, cond2, cond3))
        solver.add(Implies(cond1, And(applied_rate == 0.10, progressive_diff == 0)))
        solver.add(Implies(cond2, And(applied_rate == 0.15, progressive_diff == 1250000)))
        solver.add(Implies(cond3, And(applied_rate == 0.20, progressive_diff == 3750000)))
        solver.add(
            gift_tax_result
            == (annual_taxable_net * applied_rate)
               - progressive_diff
               - previous_gift_tax_or_credit
        )

    elif period_choice == 4:
        #
        # (4) 民國114年1月1日(含)以後
        #
        exemption_amount = 2440000
        solver.add(
            annual_taxable_net
            == annual_gift_total - exemption_amount - total_deductions
        )
        # 新稅率與累進差額規則：
        # (1) 課稅贈與淨額 ≤ 28,110,000元：10% & 0
        # (2) 28,110,001元 ≤ 課稅贈與淨額 ≤ 56,210,000元：15% & 1,405,500
        # (3) 課稅贈與淨額 ≥ 56,210,001元：20% & 4,216,000
        cond1 = (annual_taxable_net <= 28110000)
        cond2 = And(annual_taxable_net > 28110000, annual_taxable_net <= 56210000)
        cond3 = (annual_taxable_net > 56210000)
        solver.add(Or(cond1, cond2, cond3))
        solver.add(Implies(cond1, And(applied_rate == 0.10, progressive_diff == 0)))
        solver.add(Implies(cond2, And(applied_rate == 0.15, progressive_diff == 1405500)))
        solver.add(Implies(cond3, And(applied_rate == 0.20, progressive_diff == 4216000)))
        solver.add(
            gift_tax_result
            == (annual_taxable_net * applied_rate)
               - progressive_diff
               - previous_gift_tax_or_credit
        )

    else:
        raise ValueError("Unsupported period_choice")

    # ---------------------------
    # 執行求解並將結果轉換為整數
    # ---------------------------
    result_status = solver.check()
    if result_status == sat:
        model = solver.model()
        gift_tax_int = model.evaluate(ToInt(gift_tax_result))
        result = int(gift_tax_int.as_long())
        if result >= 0:
            return result
        else:
            return 0
    else:
        return None


# -----------------------------------------------
# 以下示範如何呼叫上方函式
# -----------------------------------------------
if __name__ == "__main__":

    # 範例 1: 98年舊制 (period_choice=1)
    r1 = gift_tax_calculator(
        period_choice=1,
        land_value=2000000,   # 土地
        ground_value=500000,  # 地上物
        house_value=0,
        others_value=500000,  # 合計 300萬
        not_included_land=0,
        not_included_house=0,
        not_included_others=0,
        remaining_exemption_98=2200000,
        previous_gift_sum_in_this_year=0,
        land_increment_tax=0,
        deed_tax=0,
        other_gift_burdens=0,
        previous_gift_tax_or_credit=0
    )
    print(f"[98年制計算] 應納贈與稅額= {r1} 元")

    # 範例 2: 106年 (period_choice=2)
    r2 = gift_tax_calculator(
        period_choice=2,
        land_value=30000000,
        ground_value=0,
        house_value=0,
        others_value=0,          # 本次 3000 萬
        not_included_land=0,
        not_included_house=0,
        not_included_others=0,
        remaining_exemption_98=0,
        previous_gift_sum_in_this_year=20000000,  # 之前已贈 2000 萬
        land_increment_tax=0,
        deed_tax=0,
        other_gift_burdens=0,
        previous_gift_tax_or_credit=500000,
        new_old_system_adjustment=200000
    )
    print(f"[106年制計算] 應納贈與稅額= {r2} 元")

    # 範例 3: 111年制 (period_choice=3)
    r3 = gift_tax_calculator(
        period_choice=3,
        land_value=60000000,
        ground_value=560000,
        house_value=5000000,
        others_value=4000000,
        not_included_land=0,
        not_included_house=0,
        not_included_others=0,
        remaining_exemption_98=0,
        previous_gift_sum_in_this_year=3000000,
        land_increment_tax=890000,
        deed_tax=1230000,
        other_gift_burdens=4444440,
        previous_gift_tax_or_credit=1000000,
        new_old_system_adjustment=0
    )
    print(f"[111年制計算] 應納贈與稅額= {r3} 元")

    # 範例 4: 114年以後 (period_choice=4)
    r4 = gift_tax_calculator(
        period_choice=4,
        land_value=60000000,
        ground_value=560000,
        house_value=5000000,
        others_value=4000000,
        not_included_land=0,
        not_included_house=0,
        not_included_others=0,
        remaining_exemption_98=0,
        previous_gift_sum_in_this_year=3000000,
        land_increment_tax=890000,
        deed_tax=1230000,
        other_gift_burdens=4444440,
        previous_gift_tax_or_credit=1000000,
        new_old_system_adjustment=0
    )
    print(f"[114年以後計算] 應納贈與稅額= {r4} 元")


[98年制計算] 應納贈與稅額= 80000 元
[106年制計算] 應納贈與稅額= 5220000 元
[111年制計算] 應納贈與稅額= 7961112 元
[114年以後計算] 應納贈與稅額= 7495112 元


### Selenium

In [3]:
import random
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ---------------------------
# 主程式：重複隨機測試 100 次，比對本地計算結果與網頁結果
# ---------------------------
if __name__ == "__main__":
   
    for i in range(10):
        # 隨機產生測試參數
        # period_choice 隨機選擇 1(98年)、2(106年)、3或4 (111/114年)
        period_choice = random.choice([1,2,3,4])
        params = {
            "period_choice": period_choice,
            "land_value": random.randint(0, 10000000),
            "ground_value": random.randint(0, 5000000),
            "house_value": random.randint(0, 10000000),
            "others_value": random.randint(0, 5000000),
            "not_included_land": random.randint(0, 1000000),
            "not_included_house": random.randint(0, 1000000),
            "not_included_others": random.randint(0, 1000000),
            # 根據不同規則填入：
            "remaining_exemption_98": random.randint(0, 5000000) if period_choice == 1 else 0,
            "previous_gift_sum_in_this_year": random.randint(0, 50000000) if period_choice != 1 else 0,
            "land_increment_tax": random.randint(0, 5000000),
            "deed_tax": random.randint(0, 5000000),
            "other_gift_burdens": random.randint(0, 5000000),
            "previous_gift_tax_or_credit": random.randint(0, 5000000) if period_choice in (2,3,4) else 0,
            # new_old_system_adjustment 只用於 Group 2
            "new_old_system_adjustment": random.randint(0, 500000) if period_choice == 2 else 0,
        }

        local_result = gift_tax_calculator(
            period_choice=params["period_choice"],
            land_value=params["land_value"],
            ground_value=params["ground_value"],
            house_value=params["house_value"],
            others_value=params["others_value"],
            not_included_land=params["not_included_land"],
            not_included_house=params["not_included_house"],
            not_included_others=params["not_included_others"],
            remaining_exemption_98=params["remaining_exemption_98"],
            previous_gift_sum_in_this_year=params["previous_gift_sum_in_this_year"],
            land_increment_tax=params["land_increment_tax"],
            deed_tax=params["deed_tax"],
            other_gift_burdens=params["other_gift_burdens"],
            previous_gift_tax_or_credit=params["previous_gift_tax_or_credit"],
            new_old_system_adjustment=params["new_old_system_adjustment"]
        )

        print(f"第{i+1}次，本地計算結果: {local_result}, 參數: {params}")

第1次，本地計算結果: 1032846, 參數: {'period_choice': 3, 'land_value': 9412311, 'ground_value': 1993222, 'house_value': 3999477, 'others_value': 1990496, 'not_included_land': 441222, 'not_included_house': 727188, 'not_included_others': 950354, 'remaining_exemption_98': 0, 'previous_gift_sum_in_this_year': 20863545, 'land_increment_tax': 186126, 'deed_tax': 256655, 'other_gift_burdens': 4567256, 'previous_gift_tax_or_credit': 2338506, 'new_old_system_adjustment': 0}
第2次，本地計算結果: 4884041, 參數: {'period_choice': 2, 'land_value': 6660223, 'ground_value': 753552, 'house_value': 3775517, 'others_value': 4156688, 'not_included_land': 799744, 'not_included_house': 473081, 'not_included_others': 389026, 'remaining_exemption_98': 0, 'previous_gift_sum_in_this_year': 40852181, 'land_increment_tax': 2872464, 'deed_tax': 3543237, 'other_gift_burdens': 2614497, 'previous_gift_tax_or_credit': 284806, 'new_old_system_adjustment': 326347}
第3次，本地計算結果: 0, 參數: {'period_choice': 1, 'land_value': 1392672, 'ground_value'